In [5]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from time import sleep

### Train Data Set

In [2]:
# KOBIS 연도별 영화 데이터 읽어오기 
datas = pd.read_excel('KOBIS_개봉일람_2024-04-17.xlsx', engine='openpyxl', header=6, index_col=0)
datas.head()

,곡녀,정인철,대성필림,Unnamed: 4,시네마 뉴원,2022-03-31 00:00:00,개봉영화,장편,한국,7,1616200,181,1214500,123,드라마,12세이상관람가,독립/예술영화
1,,,,,,,,,,,,,,,,,
2,극장판 시그널,하시모토 하지메,NaN,(주)미디어캐슬,트윈플러스파트너스(주),2022-03-31,개봉영화,장편,일본,90,61115200.0,10041,31131900.0,5105,드라마,12세이상관람가,독립/예술영화
3,더 맛있는 남의 떡,강현성,(주)샤이커뮤니케이션즈,NaN,(주)샤이커뮤니케이션즈,2022-03-31,개봉영화,장편,한국,1,100000.0,20,0.0,0,성인물(에로),청소년관람불가,일반영화
4,삽입하기 좋은 날,강현성,(주)샤이커뮤니케이션즈,NaN,(주)샤이커뮤니케이션즈,2022-03-31,개봉영화,장편,한국,1,100000.0,20,0.0,0,성인물(에로),청소년관람불가,일반영화
5,여대생 명기가 따로없네,김모리,영화사 레드카펫,NaN,주식회사 루믹스미디어,2022-03-31,개봉영화,장편,한국,1,1000.0,1,0.0,0,성인물(에로),청소년관람불가,일반영화
6,역할들,연송하,쉼표,NaN,쉼표,2022-03-31,개봉영화,장편,한국,5,12200000.0,2011,7007000.0,968,드라마,12세이상관람가,독립/예술영화


In [3]:
datas.columns = ['영화명', '감독', '제작사', '수입사', '배급사', '개봉일', '영화유형', '영화형태', '국적', '전국 스크린수', '전국 매출액', '전국 관객수', '서울 매출액', '서울 관객수', '장르', '등급', '영화구분']
datas.head()

,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
1,,,,,,,,,,,,,,,,,
2,극장판 시그널,하시모토 하지메,NaN,(주)미디어캐슬,트윈플러스파트너스(주),2022-03-31,개봉영화,장편,일본,90,61115200.0,10041,31131900.0,5105,드라마,12세이상관람가,독립/예술영화
3,더 맛있는 남의 떡,강현성,(주)샤이커뮤니케이션즈,NaN,(주)샤이커뮤니케이션즈,2022-03-31,개봉영화,장편,한국,1,100000.0,20,0.0,0,성인물(에로),청소년관람불가,일반영화
4,삽입하기 좋은 날,강현성,(주)샤이커뮤니케이션즈,NaN,(주)샤이커뮤니케이션즈,2022-03-31,개봉영화,장편,한국,1,100000.0,20,0.0,0,성인물(에로),청소년관람불가,일반영화
5,여대생 명기가 따로없네,김모리,영화사 레드카펫,NaN,주식회사 루믹스미디어,2022-03-31,개봉영화,장편,한국,1,1000.0,1,0.0,0,성인물(에로),청소년관람불가,일반영화
6,역할들,연송하,쉼표,NaN,쉼표,2022-03-31,개봉영화,장편,한국,5,12200000.0,2011,7007000.0,968,드라마,12세이상관람가,독립/예술영화


In [9]:
# 필요한 컬럼만 추출
# datas = datas[['영화명', '감독', '배급사', '개봉일', '전국 스크린수', '전국 매출액', '전국 관객수', '장르', '등급']]
# datas.head()

#### 추가로 필요한 데이터 - 크롤링 진행

In [27]:
# 드라이버 버전 업데이트에 문제가 있어 추가한 코드입니다 무시하셔도 됩니다!
# pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [28]:
# 드라이버 버전 업데이트에 문제가 있어 추가한 코드입니다 무시하셔도 됩니다!
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(ChromeDriverManager().install())

## 실행 가능 코드
- 페이지 접근 
- 2000년 ~ 2022년 선택 후 조회 버튼까지 누르는 코드

In [8]:
# 페이지 접근
kobis = "https://www.kobis.or.kr/kobis/business/stat/offc/searchOfficHitTotList.do?searchMode=year"
driver = webdriver.Chrome()
driver.get(kobis)
driver.maximize_window()

# 페이지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))

# 주연배우 리스트 생성
actors = []


# 조회 시작 년도 설정
year_from = driver.find_element(By.CSS_SELECTOR, '#standYySt')
select = Select(year_from)
select.select_by_value('2000')

# 조회 종료 년도 설정
year_to = driver.find_element(By.CSS_SELECTOR, '#standYyEd')
select2 = Select(year_to)
select2.select_by_value('2022')

# 조회버튼 클릭
driver.find_element(By.CSS_SELECTOR, '#searchForm > div > div.wrap_btn > button').click()

sleep(5)
driver.quit()

In [21]:
# 페이지 접근
kobis = "https://www.kobis.or.kr/kobis/business/stat/offc/searchOfficHitTotList.do?searchMode=year"
driver = webdriver.Chrome('./chromedriver.exe')
driver.get(kobis)

# 페이지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'td_movie')))

# 주연배우 리스트 생성
actors = []

# 총 19,255개의 영화
# 주연배우(3명) 크롤링해서 actor 리스트에 append 해야함
# for 

datas['주연배우'] = actors

TimeoutException: Message: 


In [40]:
datas.to_csv('movies_train_data.csv')

In [41]:
datas = pd.read_csv('./movies_train_data.csv', index_col=0)
datas

,영화명,개봉일,관객수,배급사,관람등급,상영시간,장르,주연배우수
0,명량,2014-07-30,17616299,(주)씨제이이엔엠,15세이상관람가,128분,"사극, 액션",3
1,극한직업,2019-01-23,16266480,(주)씨제이이엔엠,15세이상관람가,111분,코미디,5
2,신과함께-죄와 벌,2017-12-20,14414658,롯데쇼핑㈜롯데엔터테인먼트,12세이상관람가,139분,"판타지, 드라마",4
3,국제시장,2014-12-17,14265682,(주)씨제이이엔엠,12세이상관람가,126분,드라마,4
4,어벤져스: 엔드게임,2019-04-24,13977602,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,180분,"액션, SF",12
...,...,...,...,...,...,...,...,...
195,82년생 김지영,2019-10-23,3679265,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,118분,드라마,3
196,분노의 질주: 더 익스트림,2017-04-12,3659683,유니버설픽쳐스인터내셔널 코리아(유),15세이상관람가,136분,"액션, 범죄, 스릴러",7
197,분노의 질주: 홉스&쇼,2019-08-14,3657627,유니버설픽쳐스인터내셔널 코리아(유),12세이상관람가,135분,액션,4
198,모가디슈,2021-07-28,3619496,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,121분,"액션, 드라마",6
